In [1]:
import pandas as pd

# Recargamos el dataset normalizado con score
df_normalized = pd.read_csv('emg_normalized_labeled_score.csv')

# Definimos los umbrales
umbral = 0.3
cols_analisis = ['maximum_e1', 'standard_deviation_e1', 'root_mean_square_e1', 
                 'zero_crossings_e1', 'wave_form_length_e1', 'willison_amplitude_e1']

def auditar_fila(fila):
    criterios_cumplidos = sum(fila[col] <= umbral for col in cols_analisis)
    
    # Calculamos el score esperado según los criterios
    if criterios_cumplidos >= 5:
        score_esperado = 3
    elif criterios_cumplidos >= 3:
        score_esperado = 2
    elif criterios_cumplidos >= 1:
        score_esperado = 1
    else:
        score_esperado = 0
    
    validacion = '✔️' if fila['gb_score'] == score_esperado else '❌'
    
    return pd.Series([criterios_cumplidos, score_esperado, fila['gb_score'], validacion],
                     index=['criterios_cumplidos', 'gb_score_esperado', 'gb_score_asignado', 'validacion'])

# Aplicamos la auditoría a todo el dataset
df_auditoria = df_normalized.apply(auditar_fila, axis=1)

# Unimos con las columnas clave del dataset original
df_auditado = pd.concat([df_normalized[['gesture'] + cols_analisis], df_auditoria], axis=1)

# Guardamos el resultado en CSV
df_auditado.to_csv('emg_auditado.csv', index=False)

print("✅ Auditoría completa guardada como 'emg_auditado.csv'")


✅ Auditoría completa guardada como 'emg_auditado.csv'


In [2]:
# Conteo de validaciones
conteo_validacion = df_auditado['validacion'].value_counts()

# Mostrar resultado
print(conteo_validacion)

# Porcentaje de inconsistencias
inconsistencias = conteo_validacion.get('❌', 0)
porcentaje_inconsistencia = (inconsistencias / len(df_auditado)) * 100

print(f"📊 Inconsistencias detectadas: {inconsistencias} de {len(df_auditado)} filas ({porcentaje_inconsistencia:.2f}%)")


validacion
✔️    6822
Name: count, dtype: int64
📊 Inconsistencias detectadas: 0 de 6822 filas (0.00%)
